# Load and use model in Redis AI

1. docker run -p 6379:6379 -it --rm redisai/redisai


* https://oss.redislabs.com/redisai/commands/
* https://oss.redislabs.com/redisai/

Use Redisai.py to upload the model to Redis.

In [ ]:
import redisai
import ml2rt
from PIL import Image
from torchvision import transforms
import numpy as np

class Object(object): pass

In [ ]:
tr = Object()
tr.redis = redisai.Client()

In [ ]:
tr.loaded_model = ml2rt.load_model('plant.pt')

In [ ]:
tr.redis.modelset(
    "plant", 
    redisai.Backend.torch,
    redisai.Device.cpu,
    tr.loaded_model)

Put your images into ../input/plant-pathology-2020-fgvc7 or change the paths below

In [ ]:
!ls ../input/plant-pathology-2020-fgvc7/test/scab

In [ ]:
input_path = "../input/plant-pathology-2020-fgvc7"
tr.img = Image \
    .open(input_path + '/test/scab/Train_48.jpg') \
    .convert("RGB")
tr.img

Our model expects images in a certain format:
   * 224 x 224
   * Colors re-scaled the mean and std dev:
     * mean=[0.485, 0.456, 0.406]
     * std=[0.229, .224, .225]
   * The image must also be a part of an array, even if it's just one image. We will insert another dimension as the beginning to achieve this.

In [ ]:
tr.transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, .224, .225])
])

In [ ]:
tr.img_t = tr.transform(tr.img)
tr.img_t = np.expand_dims(tr.img_t, axis=0)
tr.img_t.shape

Send the image to Redis AI:

In [ ]:
tr.redis.tensorset('input', tr.img_t)

Apply the model to the image:

In [ ]:
tr.redis.modelrun('plant', 'input', 'pred')

Get the prediction back:

In [ ]:
pred = tr.redis.tensorget('pred')

This is prediction is from the Linear layer. To convert to probabilities, we need to apply the Softmax function:

In [ ]:
def softmax(v):
    return np.exp(v) / np.sum(np.exp(v))

In [ ]:
softmax(pred)

Our labels are:
```
['healthy', 'multiple_diseases', 'rust', 'scab']
```

### Twitter Loop

In [ ]:
import io, urllib.request, time, twitter, os, PIL

def twitter_loop():
    msg = Object()
    msg.running = True
    
    api = twitter.Api(
        consumer_key=os.environ['TWITTER_CONSUMER_KEY'],
        consumer_secret=os.environ['TWITTER_CONSUMER_SECRET'],
        access_token_key=os.environ['TWITTER_TOKEN_KEY'],
        access_token_secret=os.environ['TWITTER_TOKEN_SECRET'])   
    
    labels = ['healthy', 'multiple_diseases', 'rust', 'scab']
    
    timeline = api.GetUserTimeline(screen_name="alexonsoftware", 
                        count=10, 
                        include_rts=False,
                        exclude_replies=True)    
    
    since_id = since_id = timeline[0].id \
        if len(timeline) > 0 \
        else 0

    while msg.running:
        msg.t = api.GetUserTimeline(screen_name="alexonsoftware", 
                            count=10, 
                            include_rts=False,
                            exclude_replies=True,
                            since_id=since_id)
        if len(msg.t) > 0:
            for msg.tweet in msg.t:
                if 'media' in msg.tweet.AsDict():
                    msg.media_list = msg.tweet.AsDict()['media']
                    for msg.pic in msg.media_list:
                        msg.img = PIL.Image.open(
                            io.BytesIO(
                                urllib.request.urlopen(
                                    msg.pic['media_url']).read())).convert('RGB')
                        msg.img_t = tr.transform(msg.img).unsqueeze(0)
                        tr.redis.tensorset('image', msg.img_t.numpy())
                        tr.redis.modelrun('plant', 'image', 'pred')

                        msg.pred = tr.redis.tensorget('pred')[0]
                        msg.pred = softmax(msg.pred)
                        print(msg.pred)
                        msg.ind = range(len(labels))
                        print(msg.ind)
                        result = ''
                        for msg.p in msg.ind:
                            result += f"{labels[msg.p]} : {msg.pred[msg.p]}\n"
                        print(result)
                        api.PostUpdate(
                            status=result,
                            in_reply_to_status_id=msg.tweet.id
                        )
                since_id = max(since_id, msg.tweet.id)
        time.sleep(1)
        msg.running = True

In [ ]:
twitter_loop()